<a href="https://colab.research.google.com/github/mohan9345684510/Data-science-and-Machine-Learning-Project/blob/main/Crop_yeild.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Dependecies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
Crop_yield = pd.read_csv('/content/crop_yield.csv')
Crop_yield

In [ ]:
Crop_yield.isnull().sum()

In [ ]:
Crop_yield.columns

In [ ]:
Crop_yield['Crop'].value_counts()

In [ ]:
Crop_yield['State'].value_counts()

In [ ]:
Crop_yield['Crop_Year'].value_counts()

In [ ]:
Crop_yield['Season'].value_counts()

In [ ]:
# Import binary Encoding

!pip install category_encoders

In [ ]:
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['Crop','State','Season','Crop_Year'])
encoded = encoder.fit_transform(Crop_yield)
Crop_yield = encoded
print(Crop_yield)

In [ ]:
Crop_yield

In [ ]:
Crop_yield.shape

In [ ]:
from sklearn.model_selection import train_test_split

X = Crop_yield.drop('Yield',axis=1)
Y = Crop_yield['Yield']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_test
#print(X_train)
#print(Y_test)
#print(Y_train)

In [ ]:
Y_test.head()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
gbr.fit(X_train, Y_train)

In [ ]:
y_pred_test = gbr.predict(X_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score


mse = mean_squared_error(Y_test, y_pred_test)
r2 = r2_score(Y_test, y_pred_test)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

In [ ]:
y_pred_train = gbr.predict(X_train)

mse = mean_squared_error(Y_train, y_pred_train)
r2 = r2_score(Y_train, y_pred_train)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, Y_train)

# Best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {-grid_search.best_score_}")

In [ ]:
# Use the best estimator to make predictions
best_gbr = grid_search.best_estimator_
#print('best_gbr')
y_pred = best_gbr.predict(X_test)

# Evaluate the model
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

print(f"Mean Squared Error with Best Parameters: {mse}")
print(f"R2 Score with Best Parameters: {r2}")


In [ ]:
# Building a predictive System

input_data =(0	,0	,1	,0	,1	,0	,0	,0	,0	,1	,1	,0	,1	,1	,0	,1	,0	,1,	0	,8270.0,10280	,1484.3	,8.776951e+05	,2232.9000)

# Changing input_data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Reshape the array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = best_gbr.predict(input_data_reshaped)
print(prediction)

print('Crop Yield',prediction[0])

In [ ]:
# Compare actual values and predicted value in plot

Y_test = list(Y_test)
plt.plot(Y_test,color='red',label='Actual Value')
plt.plot(y_pred,color='green',label='Predicted Value')
plt.title('Autual value vs predicted value')
plt.legend()
plt.show()